In [126]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

from tqdm.notebook import tqdm
import numpy as np

from src.deep import file_methods
from src.optics.channel_simulation import ChannelSimulator
from src.optics.split_step_fourier import SplitStepFourier

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [127]:
# config
data_len = 50  # for each mu
num_symbols = 512
dir='data/qam1024_50x16'
# mu = 1e-3
# mu_vec = [1e-3, 1e-2, 1e-1, 0.5, 0.9]
mu_vec = np.linspace(start=0.0005,stop=0.07,num=16)
cs = ChannelSimulator(m_qam=1024,
                      num_symbols=num_symbols,
                      normalization_factor=0,  # will be overwritten during runtime
                      dt=1,
                      ssf=SplitStepFourier(
                          b2=-20e-27,
                          gamma=0.003,
                          t0=125e-12,
                          dt=1,
                          z_n=1000e3,
                          h=200
                      ),
                      verbose=False)

In [128]:
# generate the date
file_methods.gen_data(data_len, num_symbols, mu_vec, cs, dir, tqdm=tqdm)

  0%|          | 0/800 [00:00<?, ?it/s]